In [1]:
from naginterfaces.library import opt
from naginterfaces.library import rand
import numpy as np
from naginterfaces.base import utils
import warnings

# Model-based derivative free optimization

We assume that the problems are of the form 
\begin{equation*} 
\min_{x\in \mathbb{R}^n}{f(x)} 
\end{equation*}

where $f$ is a nonlinear smooth objective function which might be also
- a black box in which the derivatives are unknown,
- expensive to evaluate,
- potentially noisy

Such problems are best tackled by DFO solvers. It should be emphasized that whenever the derivatives are available and are easy to compute, a derivative-based method should be preferred. 

There are several families of DFO algorithm, we will focus on model-based methods. They rely on quadratic models that match the value of the objective function $f$ on a set of interpolation points and on a trust region method which monitors a region in which the model is deemed accurate and thus ensures the convergence. A brief description of such an algorithm can be stated as follows:

- Initialization
    * Choose a first iterate and an initial trust region around it
    * Choose a set of interpolation points inside the trust region
    * Build a quadratic interpolation model inside the trust region
- Loop until convergence is reached
    * Minimize the model in the trust region
    * Evaluate the objective on the new point
    * If the model predicted correctly the decrease of the objective:
         - replace a far away interpolation point by the new point
         - move the trust region around the new point
    * Otherwise
         - either replace some interpolation points if their geometry is not good
         - or shrink the trust region

The following animation shows 2 iterations of a model-based DFO algorithm

In [2]:
import io
import base64
from IPython.display import HTML
video = io.open('animation.mp4', 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii')))

## DFO vs a derivative-based solver using finite difference estimations for the gradient 

DFO should use a bit less function evaluations and be much more robust with regard to noise. Let's illustrate these points on four functions from the CUTEst test set:
- HART6
- ENGVAL2
- HATFLDC

Start by defining the problem characteristics:

In [3]:
def problem_char(pname):
    n = None
    xstart = None
    target = None
    # From a problem name pname, return:
    # number of variables, starting point, optimal value of the objective, lower bound, upper bound
    if pname == 'HART6':
        n = 6
        xstart = [0.2]*n
        target = -3.32288689
        xl = [0.0]*n
        xu = [1.0]*n
        
    elif pname == 'ENGVAL2':
        n = 3
        xstart = [1.0, 2.0, 0.0]
        target = 0.0
        xl = [-1.0e20]*n
        xu = [1.0e20]*n
    
    elif pname == 'HATFLDC':
        n = 25
        xstart = [0.9]*n
        target = 0.0
        xl = [0.0]*n
        xu = [10.0]*n
        
    else:
        print('problem name not known')
    return n, xstart, target, xl, xu

Initialize the RNG, the user data to record the solver progression and define the objective callback:

In [4]:
# initialize the NAG random number generator
seed = [42]
genid = 1
statecomm = rand.init_repeat(genid, seed)

In [5]:
class usr_data:
    def __init__(self, fun, statecomm=None, target=0.0, tol = 1.0e-08):
        self.fun = fun
        self.nf = 0
        self.noiselev = 0.0
        self.statecomm = statecomm
        self.pdata = None
        self.tol = tol
        self.nf = 0
        self.ok = False
        self.target = target
        self.sol = 1.0e20
        
        
class hart6_data:
    def __init__(self):
        self.c = [1.0, 1.2, 3.0, 3.2]
        self.a = [[10.0, 0.05, 3.0, 17.0],
                  [0.05, 10.0, 3.5, 8.0],
                  [17.0, 17.0, 1.7, 0.05],
                  [3.5, 0.1, 10.0, 10.0],
                  [1.7, 8.0, 17.0, 0.1],
                  [8.0, 14.0, 8.0, 14.0]]
        self.p = [[0.1312, 0.2329, 0.2348, 0.4047],
                  [0.1696, 0.4135, 0.1451, 0.8828],
                  [0.5569, 0.8307, 0.3522, 0.8732],
                  [0.0124, 0.3736, 0.2883, 0.5743],
                  [0.8283, 0.1004, 0.3047, 0.1091],
                  [0.5886, 0.9991, 0.665, 0.0381]]

Define The objective function callback. The function solved is defined in the user data structure

In [6]:
def objfun(x, inform, data=None):
    inform = 0
    n = len(x)
    obj = 0.0
    
    # Compute the objective
    if data.fun == 'HART6':
        c = data.pdata.c
        a = data.pdata.a
        p = data.pdata.p
        for i in range(len(c)):
            aux = 0.0
            for j in range(len(a)):
                aux -= a[j][i] * (x[j] - p[j][i])**2
            obj -= c[i]*np.exp(aux)
            
    elif data.fun == 'ENGVAL2':
        obj = (x[0]**2 + x[1]**2 + x[2]**2 - 1)**2 + \
        (x[0]**2 + x[1]**2 + (x[2]-2)**2 - 1)**2 + \
        (x[0] + x[1] + x[2] - 1)**2 +\
        (x[0] + x[1] - x[2] + 1)**2 +\
        (3*x[1]**2 + x[0]**3 +\
        (5*x[2] - x[0] + 1)**2 - 36)**2
        
    elif data.fun == 'HATFLDC':
        obj = (x[0]-1)**2
        for i in range(1, n-1):
            obj += (x[i+1] - x[i]**2)**2
        obj += (x[n-1]-1)**2

    else:
        print('Unknown objective function')
        inform = -1
        
    # Add noise if required
    if data.noiselev > 0.0:
        noise = rand.dist_uniform(1,-data.noiselev, data.noiselev, data.statecomm)
        obj += noise[0]
        
    # count the number of evaluations if not solved yet
    #print(abs(obj-target), obj, target)
    if not data.ok and abs(obj-data.target) < data.tol:
        data.ok = True
        data.sol = obj
        data.nf += 1
        inform = -2
    elif not data.ok:
        data.nf +=1
        
    return (obj, inform) 

# objective for the derivative based solver
def objfun_bob(x, data):
    inform = 0
    obj, inform = objfun(x, inform, data)
    return obj

Function calling the solver *bounds_quasi_func_easy*, based on gradient estimation with finite differences.

In [7]:
def solve_fd(pname, noiselevel=0.0, statecomm=None , tol=1.0e-08):
    
    n, xstart, target, xl, xu = problem_char(pname)
    data = usr_data(pname, target=target, statecomm=statecomm, tol=tol)
    if pname == 'HART6':
        h6dat = hart6_data()
        data.pdata = h6dat
    data.fun = pname
    data.noiselev = noiselevel
    print(data.fun)
    try:
        _ = opt.bounds_quasi_func_easy(ibound, objfun_bob, xl, xu, xstart, data=data)
    except utils.NagValueError as e:
        if e.errno == 2 and not data.ok:
            print('Maximum number of evaluations exceeded:', 400*n)
        elif e.errno != 2:
            print('error number not expected: ', e.errno)
    if data.ok:
        print('Final objective value', data.sol)
        print('objective evaluations', data.nf, '\n')
    else:
        print('solution not within the tolerance', data.tol, 'of the solution\n')

Function calling the derivative free solver *handle_solve_dfno*. 

In [8]:
warnings.simplefilter('error', utils.NagCallbackTerminateWarning)

def solve_dfo(pname, noiselevel=0.0, statecomm=None, options=None, tol=1.0e-08):
    iom = utils.FileObjManager(locus_in_output=False)
    # Initialize the user data with the problem name, the noise level and the convergence tolerance
    n, xstart, target, xl, xu = problem_char(pname)
    data = usr_data(pname, statecomm=statecomm, target=target, tol=tol)
    if pname == 'HART6':
        h6dat = hart6_data()
        data.pdata = h6dat
    data.fun = pname
    data.noiselev = noiselevel
    # Initialize the NAG handle data structures
    handle = opt.handle_init(n)
    # Define a nonlinear objective function in the model
    opt.handle_set_nlnobj(handle, idxfd=list(range(1, n+1)))
    # Define the bounds on the variables 
    opt.handle_set_simplebounds(handle, xl, xu)
    # Set the options
    for optstr in options:
        opt.handle_opt_set(handle, optstr)
    # Call the solver
    try:
        _ = opt.handle_solve_dfno(handle, objfun, xstart, data=data, io_manager=iom)
    except utils.NagCallbackTerminateWarning as _e:
        pass
    # print results
    print(pname)
    print('Final objective value', data.sol)
    print('Objective evaluations', data.nf)
    print()
    opt.handle_free(handle)

### No random noise in the function evaluations

First, solve these 4 problems without noise with the derivative based solver bounds_quasi_func_easy. The problems are declared solved if the solver finds a point within $10^{-8}$ of the actual solution.

In [9]:
print('No noise, derivative based solver')
print('---------------------------------')
ibound = 0

solve_fd('HART6')
solve_fd('ENGVAL2')
solve_fd('HATFLDC')

No noise, derivative based solver
---------------------------------
HART6
Final objective value -3.3228868913265734
objective evaluations 115 

ENGVAL2
Final objective value 1.5541394508620684e-09
objective evaluations 136 

HATFLDC
Final objective value 8.298144029725037e-09
objective evaluations 372 



Without noise, the solver correctly converges to the minimum.

Now let's do the same thing with the DFO solver handle_solve_dfno:

In [10]:
warnings.simplefilter('ignore', utils.NagAlgorithmicMajorWarning)
warnings.simplefilter('ignore', utils.NagAlgorithmicWarning)
options = ['Print file = -1',
           'DFO Max Objective Calls = 5000',
           'DFO Maximum Slow Steps = 1000']

print('No noise, derivative free solver')
print('--------------------------------')

solve_dfo('HART6', options=options)
solve_dfo('ENGVAL2', options=options)
solve_dfo('HATFLDC', options=options)

No noise, derivative free solver
--------------------------------
HART6
Final objective value -3.322886885382111
Objective evaluations 91

ENGVAL2
Final objective value 3.130098898053424e-09
Objective evaluations 119

HATFLDC
Final objective value 9.685746116160138e-09
Objective evaluations 373



### Minimization with some uniform noise added to the function evaluation

We now try to solve the same problems, but each time a function evaluation is requested, a random value chosen uniformely in $[-10^{-08}, 10^{-08}]$ is added. The tolerance is relaxed a bit to accomodate the noise.

The deivative-based solver gives: 

In [11]:
noiselev = 1.0e-08
tol = 1.0e-06 # relax the tolerance to be lower than the noise

solve_fd('HART6', noiselevel=noiselev, statecomm=statecomm, tol=tol)
solve_fd('ENGVAL2', noiselevel=noiselev, statecomm=statecomm, tol=tol)
solve_fd('HATFLDC', noiselevel=noiselev, statecomm=statecomm, tol=tol)
    

HART6
Maximum number of evaluations exceeded: 2400
solution not within the tolerance 1e-06 of the solution

ENGVAL2
Maximum number of evaluations exceeded: 1200
solution not within the tolerance 1e-06 of the solution

HATFLDC
Maximum number of evaluations exceeded: 10000
solution not within the tolerance 1e-06 of the solution



With even a small amount of noise added, the solver now either doesn't converge or requires a much higher number of function evaluations.

The same experiment with the DFO solver gives:

In [12]:
options = ['Print File = -1',
           'DFO Maximum Slow Steps = 1000',
           'DFO Max Objective Calls = 1500']

solve_dfo('HART6', noiselevel=noiselev, statecomm=statecomm, options=options, tol=tol)
solve_dfo('ENGVAL2', noiselevel=noiselev, statecomm=statecomm, options=options, tol=tol)
solve_dfo('HATFLDC', noiselevel=noiselev, statecomm=statecomm, options=options, tol=tol)

HART6
Final objective value -3.3228862436423685
Objective evaluations 84

ENGVAL2
Final objective value 4.481337367997389e-07
Objective evaluations 122

HATFLDC
Final objective value 9.396936939416362e-07
Objective evaluations 294



The solver still converges to a function value within the noise range for only a small cost in term of number of function evaluations

In [13]:
help(opt.handle_solve_dfno)

Help on function handle_solve_dfno in module naginterfaces.library.opt:

handle_solve_dfno(handle, objfun, x, monit=None, data=None, io_manager=None)
    Direct communication derivative-free (DFO) solver for a nonlinear
    objective function with bounded variables.
    
    Note: this function uses optional algorithmic parameters, see also:
    ``handle_opt_set``, ``handle_opt_get``.
    
    ``handle_solve_dfno`` is a forward communication Derivative-free
    Optimization (DFO) solver from the NAG optimization modelling suite
    (DFNO) for small to medium-scale nonlinear problems with bound
    constraints.
    
    For full information please refer to the NAG Library document for
    e04jd
    
    https://www.nag.com/numeric/nl/nagdoc_27/flhtml/e04/e04jdf.html
    
    Parameters
    ----------
    handle : Handle
        The handle to the problem. It needs to be initialized by
        ``handle_init`` and **must not** be changed before the call to
        ``handle_solve_dfno``.
  

In [14]:
help(opt.bounds_quasi_func_easy)

Help on function bounds_quasi_func_easy in module naginterfaces.library.opt:

bounds_quasi_func_easy(ibound, funct1, bl, bu, x, liw=None, lw=None, data=None)
    Bound constrained minimum, quasi-Newton algorithm, using function
    values only (easy-to-use).
    
    ``bounds_quasi_func_easy`` is an easy-to-use quasi-Newton algorithm
    for finding a minimum of a function F(x_1,x_2,...,x_n), subject to
    fixed upper and lower bounds of the independent variables
    x_1,x_2,...,x_n, using function values only.
    
    It is intended for functions which are continuous and which have
    continuous first and second derivatives (although it will usually
    work even if the derivatives have occasional discontinuities).
    
    For full information please refer to the NAG Library document for
    e04jy
    
    https://www.nag.com/numeric/nl/nagdoc_27/flhtml/e04/e04jyf.html
    
    Parameters
    ----------
    ibound : int
        Indicates whether the facility for dealing with bound